In [1]:
import serial
ser = serial.Serial('/dev/ttyUSB0', 115200, timeout=60*60)

In [ ]:
while x := ser.read(1):
    if x[0] == b'0xff':
        print('Device restarted or something ...')
    else:
        print(x.decode('utf-8'), end='')

Msg failed

1;1;1;0;0;0;0Position in array: 3Iteration: 4
Failed: 4
Failed to save: 0
Msg failed

1;1;1;1;0;0;0Position in array: 4Iteration: 5
Failed: 5
Failed to save: 0
Msg failed

1;1;1;1;1;0;0Position in array: 5Iteration: 6
Failed: 6
Failed to save: 0
Msg failed

1;1;1;1;1;1;0Position in array: 6Iteration: 7
Failed: 7
Failed to save: 0
Msg failed

1;1;1;1;1;1;1Position in array: 7Iteration: 8
Failed: 8
Failed to save: 0
Msg failed

1;1;1;1;1;1;1Position in array: 8Iteration: 9
Failed: 9
Failed to save: 0
Msg failed

1;1;1;1;1;1;1Position in array: 9Iteration: 10
Failed: 10
Failed to save: 0
Msg failed

1;1;1;1;1;1;1Position in array: 10Iteration: 11
Failed: 11
Failed to save: 0
Msg failed

1;1;1;1;1;1;1Position in array: 11Iteration: 12
Failed: 12
Failed to save: 0
Msg failed

1;1;1;1;1;1;1Position in array: 12Iteration: 13
Failed: 13
Failed to save: 0
Msg failed

1;1;1;1;1;1;1Position in array: 13Iteration: 14
Failed: 14
Failed to save: 0
Msg failed

1;1;1;1;1;1;1Position in arr

In [ ]:
0xff